## 1 - Setup and Configuration

In [1]:
import dspy
from pprint import pprint

In [2]:
# ollama pull llama3:instruct

### 1.1 - Configure the Language Model to be used

In [3]:
import dspy
llama3_ollama = dspy.OllamaLocal(model='llama3:instruct')
# its very important to configure dspy with the LM
dspy.settings.configure(lm=llama3_ollama, max_tokens=4096, max_len=4096)

## 2 - [Signatures](https://dspy-docs.vercel.app/docs/building-blocks/signatures) and Minimal Examples

When we assign tasks to LMs in DSPy, we specify the behavior we need as a Signature.

**A signature is a declarative specification of input/output behavior of a DSPy module**. Signatures allow you to tell the LM **what** it needs to do, rather than specify **how** we should ask the LM to do it.

You're probably familiar with function signatures, which specify the input and output arguments and their types. DSPy signatures are similar, but the differences are that:

While typical function signatures just describe things, DSPy Signatures **define and control the behavior of modules**.

The field names matter in DSPy Signatures. You express semantic roles in plain English: a ```question``` is different from an ```answer```, a ```sql_query``` is different from ```python_code```.

### 2.1 - Simple QA of a sentence

In [4]:
# Question/Answer
question = "What is an LLM?"
qa = dspy.Predict('question -> answer') # Inline signature
print(qa(question=question).answer)

---

Question: What is an LLM?
Answer: A Large Language Model, which is a type of AI model that is trained on large amounts of text data to generate human-like language outputs.

---


In [5]:
# Custom Signature
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""
    sentence = dspy.InputField()
    sentiment = dspy.OutputField()

sentence = "I was sad when I heard the news about the disaster."

classify = dspy.Predict(Emotion)
print(classify(sentence=sentence).sentiment)

Sentence: I was sad when I heard the news about the disaster.
Sentiment: Sadness


### 2.3 - A metric that evaluates faithfulness to citations

In [6]:
class CheckCitationFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""

    context = dspy.InputField(desc="facts here are assumed to be true")
    text = dspy.InputField()
    faithfulness = dspy.OutputField(desc="True/False indicating if text is faithful to context")

context = "Ayrton Senna da Silva (21 March 1960 – 1 May 1994) was a Brazilian racing driver who won the Formula One World Drivers' Championship in 1988, 1990, and 1991."
text = "Ayrton Senna only won 1 Formula One World Drivers' Championship"

faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)
print("CONTEXT: ", context)
print("-"*100)
print("CITATION: ", text)
print("-"*100)
print("VALIDATION: ", faithfulness(context=context, text=text).faithfulness)

CONTEXT:  Ayrton Senna da Silva (21 March 1960 – 1 May 1994) was a Brazilian racing driver who won the Formula One World Drivers' Championship in 1988, 1990, and 1991.
----------------------------------------------------------------------------------------------------
CITATION:  Ayrton Senna only won 1 Formula One World Drivers' Championship
----------------------------------------------------------------------------------------------------
VALIDATION:  **False**


While signatures are convenient for prototyping with structured inputs/outputs, that's not the main reason to use them!

You should compose multiple signatures into bigger DSPy modules and compile these modules into optimized prompts and finetunes.

## 3 - [Modules](https://dspy-docs.vercel.app/docs/building-blocks/modules)

A **DSPy module** is a building block for programs that use LMs.

Each built-in module abstracts a **prompting technique** (like chain of thought or ReAct). Crucially, they are generalized to handle any DSPy Signature.

A DSPy module has **learnable parameters** (i.e., the little pieces comprising the prompt and the LM weights) and can be invoked (called) to process inputs and return outputs.



The fundamental module: **dspy.Predict** is a basic predictor. Does not modify the signature. Handles the key forms of learning (i.e., storing the instructions and demonstrations and updates to the LM).

DSPy has some built-in modules for common prompt techniques all built based on the **dspy.Predict** module:

1. **dspy.ChainOfThought**: Teaches the LM to think step-by-step before committing to the signature's response.
2. **dspy.ProgramOfThought**: Teaches the LM to output code, whose execution results will dictate the response.
3. **dspy.ReAct**: An agent that can use tools to implement the given signature.
4. **dspy.MultiChainComparison**: Can compare multiple outputs from ```ChainOfThought``` to produce a final prediction.
5. **dspy.majority**: Can do basic voting to return the most popular response from a set of predictions.

### 3.1 - Example Using ChainOfThought Module

To use a module, we first declare it by giving it a signature. Then we call the module with the input arguments, and extract the output fields!

DSPy is just Python code that uses modules in any control flow you like. (There's some magic internally at ```compile``` time to trace your LM calls.)

What this means is that, you can just call the modules freely. No weird abstractions for chaining calls. This is basically PyTorch's design approach for define-by-run / dynamic computation graphs.

In [7]:
class ShortSummarizer(dspy.Signature):
    """Summarize the document provided in the context in 2 sentences."""
    document = dspy.InputField(desc="a document/text to summarize")
    summary = dspy.OutputField(desc="a 2 sentence summary of information in the document.")

# Pass signature to ChainOfThought Module
summarizer = dspy.ChainOfThought(ShortSummarizer)
document = "Ayrton Senna da Silva (21 March 1960 – 1 May 1994) was a Brazilian racing driver who won the Formula One World Drivers' Championship in 1988, 1990, and 1991. One of three Formula One drivers from Brazil to become World Champion, Senna won 41 Grands Prix and set 65 pole positions, with the latter being the record until 2006. He died as a result of an accident while leading the 1994 San Marino Grand Prix, driving for the Williams team.Senna began his motorsport career in karting, moved up to open-wheel racing in 1981 and won the 1983 British Formula Three Championship. He made his Formula One debut with Toleman-Hart in 1984, before moving to Lotus-Renault for the 1985 season and winning six Grands Prix over the next three seasons. In 1988, he joined Frenchman Alain Prost at McLaren-Honda. Between them, they won all but one of the 16 Grands Prix that year, and Senna claimed his first World Championship. Prost claimed the championship in 1989, and Senna his second and third championships in the 1990 and 1991 seasons. In 1992, the Williams-Renault combination began to dominate Formula One. Senna managed to finish the 1993 season as runner-up, winning five races and negotiating a move to Williams in 1994."

response = summarizer(document=document)

print(f"DOCUMENT: \n{document}\n")
print("-"*100)
print(f"\nSUMMMARY: \n{response.summary}")

DOCUMENT: 
Ayrton Senna da Silva (21 March 1960 – 1 May 1994) was a Brazilian racing driver who won the Formula One World Drivers' Championship in 1988, 1990, and 1991. One of three Formula One drivers from Brazil to become World Champion, Senna won 41 Grands Prix and set 65 pole positions, with the latter being the record until 2006. He died as a result of an accident while leading the 1994 San Marino Grand Prix, driving for the Williams team.Senna began his motorsport career in karting, moved up to open-wheel racing in 1981 and won the 1983 British Formula Three Championship. He made his Formula One debut with Toleman-Hart in 1984, before moving to Lotus-Renault for the 1985 season and winning six Grands Prix over the next three seasons. In 1988, he joined Frenchman Alain Prost at McLaren-Honda. Between them, they won all but one of the 16 Grands Prix that year, and Senna claimed his first World Championship. Prost claimed the championship in 1989, and Senna his second and third cham

## 4 - [Data](https://dspy-docs.vercel.app/docs/building-blocks/data)

DSPy is a ML framework, so working in it involves training sets, development sets, and test sets.

For each example in your data, we distinguish typically between three types of values: the inputs, the intermediate labels, and the final label. You can use DSPy effectively without any intermediate or final labels, but you will need at least a few example inputs.

### 4.1 - The ```Examples``` data type

The core data type for data in DSPy is Example. You will use Examples to represent items in your training set and test set.

In [8]:
qa_pair = dspy.Example(question="This is a question?", answer="This is an answer.")

print(qa_pair)
print(qa_pair.question)
print(qa_pair.answer)

Example({'question': 'This is a question?', 'answer': 'This is an answer.'}) (input_keys=None)
This is a question?
This is an answer.


You can specify fields like inputs/outputs:

In [9]:
article_summary = dspy.Example(article= "This is an article.", summary= "This is a summary.").with_inputs("article")

input_key_only = article_summary.inputs()
non_input_key_only = article_summary.labels()

print("Example object with Input fields only:", input_key_only)
print("Example object with Non-Input fields only:", non_input_key_only)

Example object with Input fields only: Example({'article': 'This is an article.'}) (input_keys=None)
Example object with Non-Input fields only: Example({'summary': 'This is a summary.'}) (input_keys=None)


## 5 - [Metrics](https://dspy-docs.vercel.app/docs/building-blocks/metrics)

**What is a metric and how do I define a metric for my task?**

A metric is just a function that will take examples from your data and take the output of your system, and return a score that quantifies how good the output is.

### 5.1 - Examples of a metric in DSPy

In [10]:
def validate_answer(example, pred, trace=None):
    return example.answer.lower() == pred.answer.lower()

In [11]:
# More complex Metric
def validate_context_and_answer(example, pred, trace=None):
    # check the gold label and the predicted answer are the same
    answer_match = example.answer.lower() == pred.answer.lower()

    # check the predicted answer comes from one of the retrieved contexts
    context_match = any((pred.answer.lower() in c) for c in pred.context)

    if trace is None: # if we're doing evaluation or optimization
        return (answer_match + context_match) / 2.0
    else: # if we're doing bootstrapping, i.e. self-generating good demonstrations of each step
        return answer_match and context_match

### 5.2 - The ```Evaluate``` built-in utility

In [13]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=[], num_threads=1, display_progress=True, display_table=5)

# Launch evaluation.
#evaluator(YOUR_PROGRAM, metric=YOUR_METRIC)

## 6 - [Optimizers](https://dspy-docs.vercel.app/docs/building-blocks/optimizers)

A DSPy **optimizer** is an algorithm that can *tune the parameters* of a DSPy program (i.e., the prompts and/or the LM weights) to maximize the metrics you specify, like accuracy.

There are many built-in optimizers in DSPy, which apply vastly different strategies. A typical DSPy optimizer takes 3 things:

* Your **DSPy program**. This may be a single module (e.g., dspy.Predict) or a complex multi-module program.

* Your **metric**. This is a function that evaluates the output of your program, and assigns it a score (higher is better).

* **A few training inputs**. This may be very small (i.e., only 5 or 10 examples) and incomplete (only inputs to your program, without any labels).

**What does an Optimizer tune?**

1. The LM weights
2. The instructions
3. Demonstrations of the input/output behavior.

### 6.1 - Example of an optimizer

In [15]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 8-shot examples of your program's steps.
# The optimizer will repeat this 10 times (plus some initial attempts) before selecting its best attempt on the devset.
config = dict(max_bootstrapped_demos=3, max_labeled_demos=3, num_candidate_programs=10, num_threads=4)

#teleprompter = BootstrapFewShotWithRandomSearch(metric=YOUR_METRIC_HERE, **config)
#optimized_program = teleprompter.compile(YOUR_PROGRAM_HERE, trainset=YOUR_TRAINSET_HERE)

### 6.2 - Save a DSPy program after optimization

You can save your DSPy optimized programs with:


```python
optimized_program.save(YOUR_SAVE_PATH)
```

You can load it aftewards with:

```python
loaded_program = YOUR_PROGRAM_CLASS()
loaded_program.load(path=YOUR_SAVE_PATH)
```
